In [2]:
from compyle.api import annotate, Elementwise, wrap, get_config
import numpy as np


@annotate
def axpb(i, x, y, a, b):
    # this is a very memory load heavy since we are doing very less flops to parallelise
    # it will give similar performance as the serial code, limited by memory bandwidth
    y[i] = a*x[i] + b


x = np.linspace(0, 1, 10000)
y = np.zeros_like(x)

# get_config().use_openmp = True  # we want to perform it for multicore architecture
get_config().use_cuda = True # for gpu architecture
# use_opencl for opencl and use_cuda for cuda
x, y = wrap(x, y)
# kernel (we want this to be run for all in parallel elementwise)
e = Elementwise(axpb)
# first arg (here x) will decide the number of elements the algorithm iterates over
e(x, y, 2.0, 3.0)

# Reduction (Like fold)

- We want to compute from all the elements into a single value
- [3, 1, 7, 0, 4, 1, 6, 3], we want to find its sum = 25 parallely
- Make a computation tree

                25
                /\
               11 14
             / \  / \
            4  7  5  9
           /\  /\  /\ /\
          3 1 7 0 4 1 6 3

- If we have $n$ numbers and $p$ processes, $\frac{n}{p}$ parallel operations, no of levels in the tree is $log_2(p)$
- Complexity is of the order of $O(\frac{n}{p} + log_2(p))$, which is close to the optimal performance we can get

# Scans 

- Generates cumulative sum
- Used for Sorting, Polynomial evaluation, Tree construction/operation, Tri-diagonal solve

- Perform an up-sweep similar to reduction of the computational tree

                25
                /\
               11 14
             / \  / \
            4  7  5  9
           /\  /\  /\ /\
          3 1 7 0 4 1 6 3

                  0
                /   \
               0     11
             / \    /   \
            0  4   11    16
           /\  /\   /\     /\
          0 3 4 11 11 15 16 22
- Down sweep will give the pre-scan
- - insert identity in root
- - left child gets node value
- - right child = left child $op$ node's value

- Complexity is of the order of $O(\frac{n}{p} + log_2(p))$, which is close to the optimal performance we can get
